<div>
<img src='assets\VFM_logo.gif' alt="Drawing" style="width: 400px; float: right;"/>
</div>

# Controleren marktdata 

- Dagelijkse controle van marktdata op afwijkingen.

  **Merijn van Miltenburg, 2020


# Inhoudsopgave

- [Inleiding](#inleiding)
- [Inlezen van de rates](#inlezen)
- [Controleren op Null Waarden](#controleren)
- [Expand Time series](#expand)
- [Berekenen van Outliers](#outliers)
- [Toevoegen Dimensions](#dimensions)
- [Definieer uitzonderingen](#uitzonderingen)
- [Presentatie van resultaten (Plotly - Dash)](#presentatie)

<a id='inleiding'></a>
# Inleiding

- De dataset kan worden gesplits in verschillende groepen
    * Yield heeft betrekking op rente curves. Dit zijn rente curves die worden gebruikt voor de waardering van de derivaten (afgeleide financiele instrumenten).     
    * Capital Price zijn prijzen van beursgenoteerde obligaties.     
    * Futures betreft marktdata van termijn contracten. 
    * Points slaan op de FRA punten (Forward Rate Agreements). Dit zijn niet beursgenoteerde rente termijncontracten. 
    * FX staat voor Foreign Exchange. Dit zijn wisselkoersen voor vreemde valuta. 
    * Dit zijn niet alle type marktdata die we dagelijks inlezen. O.a. Rentegevoeligheden (volatilities) zijn in deze dataset niet meegenomen.
    

In [3]:
import pandas as pd
import numpy as np
import datetime
from pandas.tseries.offsets import DateOffset           # Gebruikt voor het berekenen van de datum van FRA contracten

import json                                             # voor het inlezen van dimensies
from IPython.core.display import HTML                   # Gebruikt voor opmaak van print statement

import plotly.express as px                             # Voor jupyter dash
from jupyter_dash import JupyterDash                    # Voor jupyter dash
import dash_core_components as dcc                      # Voor jupyter dash
import dash_html_components as html                     # Voor jupyter dash
import dash_table as dct                                # Voor jupyter dash
from dash.dependencies import Input, Output, State      # Voor jupyter dash
from dash import callback_context                       # Voor jupyter dash
import os

<a id='inlezen'></a>
# Inlezen van de rates 


In [4]:

def read_data():
    '''
    Lees de te controleren marktdata in.
    In productie wordt dit via SQL uit het bron systeem gelezen.
    
    Voor de demo heb ik de data eerst gedownload naar CSV en lezen we hier de CSV in.
    Deze data is gemanipuleerd. De oorspronkelijke data is helaas niet publiek toegankelijk.
    De data is alleen bedoelt voor demo doeleinden.
        
    Input: 
      -        
    Output: 
      Pandas Dataframe    
    '''            
    try:
    
        ROOT_DIR = os.path.dirname(os.path.abspath("LICENCE.TXT"))
        
        # Datum kolommen
        date_cols = ["rate_dt","input_dt","actual_dt",'start_term_dt','maturity_term_dt']
    
        # Omzetten datum kolommen. int_basis en exchange name zijn gewoon string, maar werden niet goed herkend
        df = pd.read_csv(ROOT_DIR + r'\data\rates_data.csv', 
                         parse_dates = date_cols, 
                         dtype ={"int_basis":'S10', "exchange_name": 'S10'})

        # Verwijder kolommen die we niet nodig hebben
        df = df.drop(
            ['seq_no', 
             'date_days',
             'int_basis',
             'int_days',
             'start_term_dt',
             'maturity_term_dt',
             'start_term_date_days',
             'maturity_term',
             'maturity_term_date_days',
             'exchange_name',
             'bid',
             'offer',
             'days_off'
            ],
            axis=1
        )
     
    except Exception as e:
            print("ERROR: Unable to find or access file:", e)
    
    return df

def show(d):
    '''
    Deze functie print een dataframe op een wat vriendelijkere manier.
    '''
    if type (d) == pd.core.frame.DataFrame:
        display(HTML(d.to_html(index=False))) 
    else:
        print("type not recognized " + str(type(d)))        
                

# Inlezen data
df = read_data()            
show(df.head(10))


rate_dt,rate_name,ccy,rate_type,curve_type,input_dt,time_band,actual_dt,start_term,mid
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,6 MONTHS,2021-04-05,NaN,10.12276
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,1 YEAR,2021-10-05,NaN,10.54540
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,2 YEARS,2022-10-05,NaN,16.47344
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,3 YEARS,2023-10-05,NaN,23.91724
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,4 YEARS,2024-10-07,NaN,33.07982
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,5 YEARS,2025-10-06,NaN,42.51370
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,7 YEARS,2027-10-05,NaN,59.48524
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,10 YEARS,2030-10-07,NaN,72.51032
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,20 YEARS,2040-10-05,NaN,84.86218
2020-10-01,CDS Asia Banking A,USD,End of Day,Yield,10/1/2020,30 YEARS,2050-10-05,NaN,91.15178


<a id='Controleren'></a>
# Controleren en aanvullen Null Waarden

Eerste stap is de controle van de brongegevens. 

Bij Bonds en Bond Yields is de kolom actual_dt (value date van het datapunt) vaak niet ingevuld. Deze vul ik aan.
De FRA punten kennen geen waarde voor de actual_dt en een verkeerde waarde voor time_band. Deze bereken ik op basis van de rate date en de start term kolom en vul ik aan.

In [5]:
def clean_data(df):
    '''
    Opschonen van de ingelezen data
    
    '''
    # Eerst actual date vullen op basis van Rate Date
    df['actual_dt'].fillna(df['rate_dt'], inplace = True)

    # Voor FRA Points: actual_dt = rate_dt + start_term aantal maanden.
    # Ik gebruik hier itterrows (trager), omdat ik op deze wijze de date offset kan berekenen.
    dfs = df.loc[df['curve_type']=='Points']
    for i, row in dfs.iterrows():    
        new_date = (row.rate_dt + DateOffset(months=row.start_term))          
        df.iloc[i,df.columns.get_loc("actual_dt")] = new_date   
        if (row.start_term == 1):
            time_band = '1 MONTH'
        else:
            time_band = str(int(row.start_term)) + ' MONTHS'                    
        df.iloc[i,df.columns.get_loc("time_band")] = time_band
    
clean_data(df)

<a id='expand'></a>
# Expand Time Series

Nu gaan we op zoek naar de ontbrekende keys voor bepaalde data.

Iedere curve kan een verschillend aantal data punten hebben. Per curve moet worden bekeken of alle punten zijn ingelezen. 
Het aantal punten dat beschikbaar is verschilt echter per rentecurve.  In eerste instantie bouw ik een dataset met alleen de rate names, en de term points. Deze gebruik ik daarna om er zeker van te zijn dat voor alle dagen alle punten in de dataset aanwezig zijn. Wanneer een punt niet beschikbaar is wordt deze gevuld met N/A.  Om alle rates voor alle dagen beschikbaar te krijgen - wordt er een cartesian product van gemaakt door een gezamelijke kolom toe te voegen. 

yield_keys bevat uiteindelijk alle mogelijke rates, ccy's en terms voor alle mogelijke dagen.
df_expanded bevat alle rates data - inclusief 'gaten' wanneer er op sommige dagen data ontbreekt.
  
Om te zien of dit werkt heb ik in de USD Deposit curve op 2 okt een paar data punten verwijderd.

In [6]:

def expand_time_series(df, time, keys):
    '''
      Deze functie vult een dataframe met alle mogelijke combinaties van 2 dimensies.
      De functie krijgt als input een dataframe en 2 lijsten met dimensies. De eerste dimensie geeft de tijdseenheid aan. 
      De tweede dimensie geeft de sleutels per tijdseenheid mee. De functie expand_time_series voegt NaN values toe voor alle ontbrekende combinaties.
      
      Input: 
      df -              Pandas Datafame
      time -            list : Naam / namen van de kolom(men) van de tijdsdimensie
      keys -            list : Namen van de overige sleutelvelden      
      
      Output: 
      Pandas Dataframe      
    '''    
    df_time = df[time].drop_duplicates()
    df_keys = df[keys].drop_duplicates()

    # een fake kolom wordt toegevoegd om een cartesian product merge te maken
    df_time['k'] = 0 
    df_keys['k'] = 0

    df_expanded_keys = df_time.merge(df_keys, how='left')
    
    # Expanded keys wordt met een left join toegevoegd
    df_expanded = df_expanded_keys.merge(df, on=time + keys, how = 'left', indicator=True)
    
    # Verwijder hulp kolommen
    del df_expanded['k']
    del df_expanded['_merge']
    df_expanded.reindex()
    
    return df_expanded

# Toon een voorbeeld van het effect van het uitbreiden van de dataset
currency = 'USD'
rate_name = 'Deposit'
selection = (df['rate_dt']=='2-OCT-2020') & (df['ccy']==currency) & (df['rate_name']==rate_name)
columns = ['rate_dt','curve_type','rate_name','rate_type','ccy','time_band','mid']

# Before extending the data
show(df.loc[selection,columns])

df_ext = expand_time_series(df, ['rate_dt'], ['curve_type','rate_name','rate_type','ccy','time_band'] )

# After extending the data
selection = (df_ext['rate_dt']=='2-OCT-2020') & (df_ext['ccy']==currency) & (df_ext['rate_name']==rate_name)
show(df_ext.loc[selection ,columns])

rate_dt,curve_type,rate_name,rate_type,ccy,time_band,mid
2020-10-02,Yield,Deposit,End of Day,USD,OVERNIGHT,0.07594
2020-10-02,Yield,Deposit,End of Day,USD,2 DAYS,0.06820
2020-10-02,Yield,Deposit,End of Day,USD,1 WEEK,0.09332
2020-10-02,Yield,Deposit,End of Day,USD,2 WEEKS,0.08518
2020-10-02,Yield,Deposit,End of Day,USD,3 WEEKS,0.10772
2020-10-02,Yield,Deposit,End of Day,USD,1 MONTH,0.12882
2020-10-02,Yield,Deposit,End of Day,USD,1 YEAR,0.30895


rate_dt,curve_type,rate_name,rate_type,ccy,time_band,mid
2020-10-02,Yield,Deposit,End of Day,USD,OVERNIGHT,0.07594
2020-10-02,Yield,Deposit,End of Day,USD,2 DAYS,0.06820
2020-10-02,Yield,Deposit,End of Day,USD,1 WEEK,0.09332
2020-10-02,Yield,Deposit,End of Day,USD,2 WEEKS,0.08518
2020-10-02,Yield,Deposit,End of Day,USD,3 WEEKS,0.10772
2020-10-02,Yield,Deposit,End of Day,USD,1 MONTH,0.12882
2020-10-02,Yield,Deposit,End of Day,USD,2 MONTHS,NaN
2020-10-02,Yield,Deposit,End of Day,USD,3 MONTHS,NaN
2020-10-02,Yield,Deposit,End of Day,USD,4 MONTHS,NaN
2020-10-02,Yield,Deposit,End of Day,USD,5 MONTHS,NaN


<a id='outliers'></a>
# Berekenen van outliers
  
Z-score wordt gedefinieerd over 20 dagen historie om outliers te herkennen
Daarnaast wordt bp_diff (Base Point Difference) berekend ten opzichte van de vorige dag.
Rate waar geen wijziging op heeft plaatsgevonden in de laatste 20 dagen worden gezien als outlier
Ook gaten in de rates worden toegevoegd als exceptions. De exceptions worden nu alleen berekend over de laatste dag. Een mooie uitbreiding zou kunnen zijn om ook exceptions over de voorliggende dagen te berekenen.

- z-score = (rates.mid - rates.mid.mean()) / rates.mid.std()
- bp_diff = rates.mid - rates.mid(t-1) * 100
- No_change = (rates.mid = rates.mid(t-1) = rates.mid(t-n))


In [ ]:
# df_na bevat alleen de rates met ontbrekende data punten
# df_na = df_ext.loc[((df_ext['mid'].isna()) & (df_ext['rate_dt']== df_ext['rate_dt'].max()))
#                    ,columns].copy()

In [7]:
def calc_group(df):
       
    keys = ['curve_type','rate_name','rate_type','ccy','time_band']
    columns =  ['rate_dt', 'curve_type', 'rate_name', 'rate_type', 'ccy', 'time_band', 'mid']
    # Selectie van range van de laatste 20 dagen
    df_20d = df[df.rate_dt >= df['rate_dt'].unique()[-20]].copy()
        
    # Toevoegen van gemiddelde en standaard afwijking - daarna berekenen van z-score
    
    df_gr = df_20d.groupby(keys).agg( 
        {'mid' : ['mean', 'std'],'actual_dt': ['last'] } ) 
    
    # hernoemen output kolommen
    df_gr.columns = ['mean', 'std','actual_dt'] 

    # Group by levert een multi index. Deze wordt hier weer plat geslagen
    df_gr = df_gr.reset_index()
    
    # Wanneer rates op de laatste dag geen data hebben - neem ik deze niet mee.
    # Deze worden dan alleen getoond bij de NA exceptions.
    max_dt = df['rate_dt'].max()
    df_last_dt = df.loc[df['rate_dt']==max_dt,columns]
    df_gr = df_gr.merge(df_last_dt, how = "inner",on = keys)

    # Bereken nu de z-score
    df_gr['z-score'] = ( df_gr['mid'] - df_gr['mean'] ) / df_gr['std']
    
    # Concatinate the last 2 days in the dataframe
    # Dit kan simpeler met een diff functie?
    df_last = df[df['rate_dt']==df['rate_dt'].max()] # Only the latest dates
    df_prev = df[df['rate_dt']==df['rate_dt'].unique()[-2]] # Previous business day T-1
    df_diff = pd.merge(df_last,df_prev,how='left',on = keys)

    # Bereken nu bp_delta
    df_diff['bp_diff'] = (df_diff['mid_x']-df_diff['mid_y'])*100 # diff = verschil T en T-1 * 100 (in basispunten)
    df_gr = df_gr.merge(df_diff[keys + ['bp_diff']], how = "left",on = keys) # Voeg bp_diff toe aan de dataset

    # Afronden op 5 decimalen voor presenatie
    df_gr['z-score'] = df_gr['z-score'].round(5).abs()
    df_gr['bp_diff'] = df_gr['bp_diff'].round(5)
    df_gr['mid'] = df_gr['mid'].round(5)
    df['mid'] = df['mid'].round(5)
    
    return df_gr

# Calculate de z-score en bp_diff
df_gr = calc_group(df)

In [ ]:

# # Selectie van range van de laatste 20 dagen
# df_20d = df[df.rate_dt >= df['rate_dt'].unique()[-20]].copy()

# # Toevoegen van gemiddelde en standaard afwijking - daarna berekenen van z-score
# keys = ['curve_type','rate_name','rate_type','ccy','time_band']

# df_gr = df_20d.groupby(keys).agg( 
#     {'mid' : ['mean', 'std'],'actual_dt': ['last'] } ) 

# # hernoemen output kolommen
# df_gr.columns = ['mean', 'std','actual_dt'] 

# # Group by levert een multi index. Deze wordt hier weer plat geslagen
# df_gr = df_gr.reset_index() 

# # Wanneer rates op de laatste dag geen data hebben - neem ik deze niet mee.
# # Deze worden dan alleen getoond bij de NA exceptions.
# max_dt = df['rate_dt'].max()
# df_last_dt = df.loc[df['rate_dt']==max_dt,columns]
# df_gr = df_gr.merge(df_last_dt, how = "inner",on = keys)

# # Bereken nu de z-score
# df_gr['z-score'] = ( df_gr['mid'] - df_gr['mean'] ) / df_gr['std']

# # Concatinate the last 2 days in the dataframe
# # Dit kan simpeler met een diff functie
# df_last = df[df['rate_dt']==df['rate_dt'].max()] # Only the latest dates
# df_prev = df[df['rate_dt']==df['rate_dt'].unique()[-2]] # Previous business day T-1
# df_diff = pd.merge(df_last,df_prev,how='left',on = keys)

# # Bereken nu bp_delta
# df_diff['bp_diff'] = (df_diff['mid_x']-df_diff['mid_y'])*100 # diff = verschil T en T-1 * 100 (in basispunten)
# df_gr = df_gr.merge(df_diff[keys + ['bp_diff']], how = "left",on = keys) # Voeg bp_diff toe aan de dataset

# # Afronden op 5 decimalen voor presenatie
# df_gr['z-score'] = df_gr['z-score'].round(5).abs()
# df_gr['bp_diff'] = df_gr['bp_diff'].round(5)
# df_gr['mid'] = df_gr['mid'].round(5)
# df['mid'] = df['mid'].round(5)

<a id='dimensions'></a>
# Toevoegen van dimensies

        Yield group en currency groups.        
        Dit wordt gedaan om beter te kunnen selecteren bij de presentatie van de rates
            

In [8]:
# Met deze aanpak worden de hierarchien meerdere keren ingelezen.
# Dat is ook niet zo fraai...

def add_hiarchies(df):
    '''
    Deze functie voegt de dimensies toe die zijn gedefinieerd in de json data.
        
    Input: 
      df -              Pandas Datafame       
      
    Output: 
      Pandas Dataframe     
    '''    
    try:

        # Lezen van hiarchie-en vanuit JSON
        df_r = pd.read_json(r'data\ccy_group.json')
        df = pd.merge(df,df_r,how = 'left')

        # Lezen van hiarchie-en vanuit JSON
        df_r = pd.read_json(r'data\yield_group.json')
        df = pd.merge(df,df_r,how = 'left')
                
    except Exception as e:
        print("ERROR: Unable to find or access file:", e)
    
    return df

df = add_hiarchies(df)
df_gr = add_hiarchies(df_gr)
df_ext = add_hiarchies(df_ext)

In [ ]:
# try:

#     # Lezen van hiarchie-en vanuit JSON
#     df_r = pd.read_json(r'data\ccy_group.json')
#     df_gr = pd.merge(df_gr,df_r,how = 'left')
#     df_na = pd.merge(df_na,df_r,how = 'left')
#     df = pd.merge(df,df_r,how = 'left')

#     # Lezen van hiarchie-en vanuit JSON
#     df_r = pd.read_json(r'data\yield_group.json')
#     #df_r['curve_type'] = 'Yield'

#     # Merge de nieuwe dimensie bij de data
#     df_gr = pd.merge(df_gr,df_r,how = 'left')
#     df_na = pd.merge(df_na,df_r,how = 'left')
#     df = pd.merge(df,df_r,how = 'left')

# except Exception as e:
#     print("ERROR: Unable to find or access file:", e)

<a id='uitzonderingen'></a>
# Definieer de uitzonderingen 

1. Z-score te hoog of te laag (> 3.09) en Criteria op basis van basis punten
    * Bonds - 50 basis punten
    * Futures 50 basis punten
    * FX       2.5 basis punten
    * yield per tenor verschillend uit tabel
    
2. Punten die niet hebben bewogen in de laatste 20 dagen

3. Gaten in de data (Isna)


In [9]:

def define_exceptions(df):
    '''
    Outside limit Exceptions worden bepaald adhv een limiet per rate, currency en tenor
    Stale rate Exceptions worden bepaald door te kijken naar de std deviatie over de laatste 20 dagen    
    
    Input: 
      df -              Pandas Datafame       

    Output: 
      Pandas Dataframe     
    '''
    
    
    try:    
        # In deze tabel staan de Treshold waarden per ccy group, en tenor
        df_t = pd.read_excel(r'data\yield_tenor_treshold.xlsx')

        # Vertaal ccy groep in de treshold tabel naar dezelfde codering in de dataframe
        df_t['currency_group'] = df_t['ccygroup'].map({1:'A-currency',2:'B-currency',3:'C-currency'})

        # Merge treshold tabel op ccy groep en tenor ( = time_band)
        df = df.merge(df_t, how = 'left', left_on = ['ccy_group', 'time_band'], right_on = ['currency_group','timeband'] )

    except Exception as e:
        print("ERROR: Unable to find or access file:", e)


    # Set default value voor de overige tresholds
    df.loc[(df['curve_type']=='Capital Pr'),['threshold']] = 50
    df.loc[(df['curve_type']=='Futures'),['threshold']] = 50
    df.loc[(df['curve_type']=='FX'),['threshold']] = 2.5
    # Wanneer geen yield exceptie is opgegeven in de tabel - dan default 10.
    df.loc[((df['curve_type']=='Yield')),['threshold']].fillna(10)

    # outside limit exception wordt ingesteld op de gecombineerde conditie van z-score en bp_delta
    df['outside_limit_exception'] = np.where( 
        ( ( df['z-score']>3.09) | (df['z-score'] < -3.09) ) 
        & ( np.isfinite(df['z-score'] )
        & ( df['bp_diff'].abs() > df['threshold'] ) ), True, False)

    df['stale_rate_exception'] = np.where( df['std']==0 , True, False )

    # Nog leuk om toe te voegen: Wanneer is dit datapunt voor het laatst gewijzigd?
    # Dat bleek nog lastig - nog later toe te voegen...
    
    return df
    
df_gr = define_exceptions(df_gr)

In [10]:
def concat_exceptions(df_gr, df_ext):
    '''
    Samenvoegen van alle exceptions in 1 tabel voor presentatie
    Input:
        df_gr: Pandas Dataframe
        df_ext: Pandas Dataframe
        
    Output
        Pandas Dataframe
    '''
    columns = ['rate_dt','curve_type','rate_name','rate_type','ccy','time_band','mid', 'ccy_group','yield_group']
    df_outside_limit = df_gr.loc[df_gr['outside_limit_exception'], columns].copy()

    # Curve punt heeft niet bewogen in de afgelopen 20 dagen
    df_stale_rate = df_gr.loc[df_gr['stale_rate_exception'], columns].copy()

    # df_na bevat alleen de rates met ontbrekende data punten
    df_na = df_ext.loc[((df_ext['mid'].isna()) & (df_ext['rate_dt']== df_ext['rate_dt'].max()))
                       ,columns].copy()

    # Toevoegen van een exception column die het type exception beschrijft
    df_outside_limit['exception'] = 'Outside limit'
    df_stale_rate['exception'] = 'Stale Rate'
    df_na['exception'] = 'No Rate'

    # Voeg alle exceptions samen in 1 presentatie tabel
    return pd.concat([df_outside_limit,df_stale_rate,df_na])

df_exceptions = concat_exceptions(df_gr, df_ext)

In [ ]:
# #Schrijf de data even naar Excel om verder te onderzoeken
# try:
#     df.to_excel(r'c:\temp\df.xlsx', index = False)
#     df_gr.to_excel(r'c:\temp\df_gr.xlsx', index = False)
#     df_exceptions.to_excel(r'c:\temp\df_exceptions.xlsx', index = False)

# except Exception as e:
#     print ("ERROR: Unable to find or access file:", e)

    

In [11]:
# Uitfilteren van de geaccepteerde verschillen
def filter_exceptions(df, df_filter):
    '''
      filter exceptions filters de geselecteerde velden van de 2e dataset uit de eerste dataset op basis van overeenkomende velden
    
    
    Input: 
      df                Pandas Datafame: dataset to be filtered
      df_filter         Pandas Datafame: dataset to be removed   

      Output: 
      Pandas Dataframe       
    '''    

    if len(df_filter) > 0:
        dff = df.merge(df_filter, how='outer', indicator=True)
        dff = dff.query('_merge == "left_only"').drop('_merge', 1)
    else: dff = df
               
    return dff

<a id='presentatie'></a>
# Presentatie van de resultaten

De data wordt gepresenteerd in de dashboard.

1) Overzicht van de 'echte Outliers': 
    Z-score < -3.09 of Z-score > 3.09 + BP_delta > x 
    Of std van 0    
2) Tabblad per curve type

In [12]:
# Hernoemen van curve_type waarde voor bonds voor de leesbaarheid in de presentatie
df_exceptions["curve_type"].replace({'Capital Pr': 'Bonds'}, inplace=True)
df_gr["curve_type"].replace({'Capital Pr': 'Bonds'}, inplace=True)
df["curve_type"].replace({'Capital Pr': 'Bonds'}, inplace=True)


# Drop spot_end_of_day (duplicate data)
df_gr = df_gr[df_gr['rate_type']!='Spot End of Day']

# Prepare data for Tab1

# check_dt is de meest actuele datum
check_dt = df_gr['rate_dt'].max()

subline = '''
_Visual inspection of daily upload results and outliers._
''' + check_dt.strftime('%d-%m-%Y')

# Kolommen die getoond worden bij de uitzonderingen
tab1_cols = ['exception','curve_type','rate_name','rate_type','ccy','time_band','mid']
tab1_cols_alpha = ['exception','curve_type','rate_name','rate_type','ccy','time_band']

def selected_to_df(tablerows, selected_rows):
    '''
    Vertaal selectie in Dash datatable naar een pandas dataset
    
    Input:
    tablerows:     dict: data vanuit datatable object
    selected_rows: list: list van indexes van selected rows (multi-select data table)
    
    Output:
    Pandas dataframe

    '''
          
    if len(tablerows) > 0:
        
        cols = list(tablerows[0].keys())        
        
        # New dataframe
        dfi = pd.DataFrame(cols)  

        for i in selected_rows:
            row = tablerows[i]  
            rowkeys = { k: row[k] for k in cols}
            dfi = dfi.append(rowkeys,ignore_index=True)  
            
    else:
        dfi = pd.DataFrame()  
                
    
    return dfi


# Kolommen die getoond worden bij tab 2 
fx_cols = ['ccy_group', 'ccy', 'rate_type', 'mid', 'z-score', 'bp_diff']
fx_cols_alpha = ['ccy_group', 'ccy', 'rate_type']

# Kolommen die getoond worden bij tab 4 Bonds
bond_cols = ['rate_name', 'ccy', 'mid', 'z-score', 'bp_diff']
bond_cols_alpha = ['rate_name', 'ccy']

# Prepare data for Tab 3: interest Rates and FRA Points
df_ir = df.loc[df['curve_type'].isin(['Yield','Points']) ].copy()

# drop alle data punten langer dan 30 jaar
df_ir = df_ir[~df_ir['time_band'].isin(['35 YEARS','40 YEARS','45 YEARS','50 YEARS','55 YEARS','60 YEARS'])]
df_ir = df_ir.sort_values(['rate_name','rate_dt','actual_dt']).reset_index(drop=True)

# Drop yield groups Recovery Rates en Bond Yields - Geen curve data
df_ir = df_ir[~df_ir['yield_group'].isin(['Recovery Rate','Bond Yield'])]
        
# Sorteren van data voor line plots
keys = ['rate_dt','curve_type','rate_name','rate_type','ccy','time_band']
df.sort_values(by=keys, inplace=True)
df_gr.sort_values(by=keys, inplace=True)



In [13]:
# Translate ccy to ccy pair based on dominant ccy
def ccypair(ccy):
    '''
    Translate ccy to ccy pair based on dominant ccy with base ccy USD
    
    Input: 
        ccy     String
    Output:
        ccypair String
    
    '''
    if ccy in ['EUR','GBP','AUD','NZD']:
        return ccy + '/USD'
    else:
        return 'USD/'+ ccy     
    

In [14]:

def linechart(df,x,y, title, xaxis_title, yaxis_title,color=None, hover_mode=False, hover_column=None):
    '''
    Construct line graph
    Input: 
    df:           dataframe
    x:            x-axis
    y:            y-axis   
    title:        chart title
    xaxis-title   title xaxis
    xaxis-title   title yaxis
    color:        group by variable - default = None
    hover_mode    plotly hover value (x,y,closest,x unified) default is False
    hover_column  Optioneel: kolom die terug gegeven moet worden als custom data

    output:
    fig           dict

    '''
    if len(df.index) == 0:           
        fig = {}
    else:        
        if hover_column is not None:
            fig = px.line( df, x=x, y=y, color = color,
                       custom_data=[hover_column]  )
        else:
            fig = px.line( df, x=x, y=y, color = color)


        fig.update_traces(mode="markers+lines", connectgaps=True)                       

        fig.update_layout( title=title, xaxis_title=xaxis_title, yaxis_title= yaxis_title,hovermode=hover_mode,
                       xaxis_tickformat = '%d-%m-%Y' )
                
        
        
        # Custom Hover tips - helaas kon ik de koptekst niet aanpassen
        if hover_column is not None:        
            fig.update_traces(
                hovertemplate="<br>".join([            
                    "%{y}",
                    "%{customdata[0]}"        
                ])
        )        
    return fig

In [15]:
# Build Dash App

# Html Css Style sheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Build App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Dit is noodzakelijk om callsback van de tabs die nog niet gerendered zijn, te kunnen definieren
app.config['suppress_callback_exceptions'] = True

# Main application layout
app.layout = html.Div(
    [       
        html.Div( 
            [
                 html.Div( children = 
                     [
                        html.H1( "Check Marktdata " ),                  
                        dcc.Markdown(subline),     
                     ]
                     , style={'display': 'inline-block', 'width': '49%'}
                 ),
                 html.Div(                      
                     html.Img(src=app.get_asset_url('VFM_logo.gif'), style={ 'width' : 400, 'float' : 'right' } )                 
                 , style={'display': 'inline-block', 'width': '49%'}
                 )
            ]
        ),                 
        html.Br(),  
        dcc.Tabs(
            id='tabs-overview', 
            value='tab-1', 
            children=[            
                dcc.Tab(label='Exceptions', value='tab-1'),
                dcc.Tab(label='FX', value='tab-2'),
                dcc.Tab(label='Interest Yield', value='tab-3'),                
                dcc.Tab(label='Bonds', value='tab-4')                
            ]
        ),
        html.Div(
            id='tabs-overview-content'
        )
    ], style= { 'width': '100%', 'display': 'inline-block', 'padding-left':'2%', 'padding-right':'2%' }    
)
        
# Callback voor main app
@app.callback(Output('tabs-overview-content', 'children'),
              Input('tabs-overview', 'value'))
def render_content(tab):
    if tab == 'tab-1':         
        return tab1             
    elif tab == 'tab-2':
        return tab2        
    elif tab == 'tab-3':
        return tab3            
    elif tab == 'tab-4':
        return tab4               

# Render Tab1 : Exceptions data

tab1 = html.Div(
    [
        html.Div( 
            dcc.ConfirmDialog( 
                id='tab1_confirm',
                message='Are you sure you want to supress warnings for these rates?'
            )
        ),        
        html.Div(
            [   
                html.H3(
                    id = 'tab1_header',    
                    children=["Exceptions"]
                ), 
                dcc.Markdown('''
In the data below you see the outliers defined by the business. 
A data point is considered an outlier if it meets one of the following criteria. Outside Limits exceptions are shown when a data point has moved outside the limits set by the Bank. We check on both the [z-score](https://en.wikipedia.org/wiki/Standard_score) > 3.09 and daily base point value movement.
Stale Rates means the data point has not moved for at least 20 days. No Rate exceptions are shown when no rate is available for a datapoint in a curve on a tenor where we previously did have a data point. 
'''             ),  
                html.Br(),
                dcc.Checklist(
                    id = 'tab1_checklist_exceptions', 
                    options=[{'label': i, 'value': i} for i in df_exceptions['exception'].unique()], 
                    labelStyle={'display': 'inline-block'},         
                ),            
                html.Div(
                    [ 
                        dct.DataTable( 
                            id='tab1_datatable_exceptions',                             
                            row_selectable='multi', 
                            page_size=20, 
                            # Left allign de tekst kolommen                        
                            style_cell_conditional = [ { 'if': {'column_id': c}, 'textAlign': 'left' } for c in tab1_cols_alpha ],
                            # gestreepte regels voor de leesbaarheid
                            style_data_conditional=[ { 'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)' }],
                            # achtergrond kopregel
                            style_header={'backgroundColor': 'rgb(230, 230, 230)','fontWeight': 'bold' },                                            
                            # Style as list
                            style_as_list_view=True
                        )               
                    ]
                )                    
            ], style={'display': 'inline-block', 'width': '80%','padding-left':'2%', 'padding-right':'2%'}
        ),
        html.Div(
            [    
                dcc.RadioItems(
                    id='tab1_radio_show_suppressed', 
                    options= [ {'label': 'Suppressed', 'value': 1}, {'label':'Exceptions', 'value': 0}],
                    value=0,            
                    labelStyle={'display': 'inline-block'}
                ),                
                html.Br(),
                html.P( 'Hide or show the exceptions '),
                html.Button(
                    'Hide', 
                    id='tab1_hide_button', 
                    n_clicks=0
                ),            
            ], style={'display': 'inline-block','padding-left':'2%', 'padding-right':'2%'}               
        )
    ]
)

# Callback header tab1
@app.callback(
    Output('tab1_header','children'),
    Input('tab1_radio_show_suppressed','value')
)
def update_radio(show_suppressed):
    
    if show_suppressed:
        return 'Suppressed'
    else:
        return 'Exceptions'

#Callback confirm dialog : change the message depending on state
@app.callback(    
    Output('tab1_confirm','message'),
    Input('tab1_radio_show_suppressed','value')
)
def update_confirmdialog(show_suppressed):
        
    
    if show_suppressed:
        return 'unhide the selected exceptions?'
    else:
        return 'Are you sure you want to supress warnings for these rates?'
            
# call back button name    
@app.callback(
    Output('tab1_hide_button','children'),
    Input('tab1_radio_show_suppressed','value')        
)
def update_button(show_suppressed):    
        
    if show_suppressed:
        return 'Unhide'
    else:
        return 'Hide'
    
# Callback confirmation dialog - Hide exceptions warning dialog
@app.callback(    
    Output('tab1_confirm', 'displayed'),
    Input('tab1_hide_button', 'n_clicks'),        
    State('tab1_datatable_exceptions', 'selected_rows')
)
def update_out(n_clicks, selected):
            
    if selected: 
        return True
    else:
        return False
    
# Define callback for Tab1 data table content
@app.callback(Output('tab1_datatable_exceptions', 'data'),
              Output('tab1_datatable_exceptions', 'columns'),
              Output('tab1_datatable_exceptions', 'selected_rows'),
              Input('tab1_checklist_exceptions', 'value'),
              Input('tab1_confirm', 'submit_n_clicks'),
              Input('tab1_radio_show_suppressed','value'),           
              State('tab1_datatable_exceptions', "data"),
              State('tab1_datatable_exceptions', "selected_rows")              
             )
def show_data(checklist_exceptions, savebutton, show_suppressed, tablerows, selected_rows):                        
        
    if selected_rows is None:
        selected_rows = []
           
    # determine context - dwz welke input voorzaakte de call?
    ctx = callback_context
              
    if ctx.triggered:            
        # Component dat de callback aanroept
        input_component = ctx.triggered[0]['prop_id'].split('.')[0]
                    
        if input_component == 'tab1_confirm':
            
            if not selected_rows:
                print ('ERROR: Selection made with 0 items in callback tab1')
            else:                    
                if show_suppressed:

                    # de geselecteerde regels moeten worden verwijderd uit de exception lijst      
                                        
                    df_hide_exceptions = pd.DataFrame.from_dict(tablerows)                     
                    df_deleted = selected_to_df(tablerows, selected_rows)                    
                    df_hide_exceptions = filter_exceptions(df_hide_exceptions,df_deleted)

                else:                        
                    try:
                        df_hide_exceptions = pd.read_excel(r'data\exceptions.xlsx')
                                                                
                        # Voeg de geselecteerde excepties toe aan de lijst van uitzonderingen
                        for i in selected_rows:
                            row = tablerows[i]
                            rowkeys = { k: row[k] for k in ['exception','curve_type','rate_name','rate_type','ccy','time_band'] }
                            df_hide_exceptions = df_hide_exceptions.append(rowkeys,ignore_index=True)   

                    except Exception as e:
                        print("ERROR: Unable to find or access file:", e)                    
            
                try:
                    # save the hide exceptions dataframe - to disc                
                    df_hide_exceptions.to_excel(r'data\exceptions.xlsx', index = False)
                    
                    selected_rows = []

                except Exception as e:
                    print("ERROR: Unable to find or access file:", e)                        

    try:
        
        df_hide_exceptions = pd.read_excel(r'data\exceptions.xlsx') 
                    
        if show_suppressed:      
                        
            dff_hide_exceptions = df_hide_exceptions
            
            # Filter de geselecteerde exceptions
            if checklist_exceptions:
                dff_hide_exceptions = df_hide_exceptions[df_hide_exceptions['exception'].isin(checklist_exceptions)]
            
            if len(dff_hide_exceptions.index) > 0:
                data = dff_hide_exceptions[tab1_cols_alpha].to_dict('records')
            else:
                data = []
            
            columns = [{"name": i, "id": i} for i in tab1_cols_alpha]                      
           
            
        else:
              
            # Check of een of meer type excepties getoont moeten worden, en beperk zo nodig de selectie
            dff_exceptions = df_exceptions
            
            # Filter de geselecteerde exceptions
            if checklist_exceptions:
                dff_exceptions = df_exceptions[df_exceptions['exception'].isin(checklist_exceptions)]

            # Filter de uitzonderingen uit de lijst die we niet willen zien
            dff_exceptions = filter_exceptions(dff_exceptions, df_hide_exceptions)

            data = dff_exceptions[tab1_cols].to_dict('records')
            columns = [{"name": i, "id": i} for i in tab1_cols]

        
    except Exception as e:
        print("ERROR: Unable to find or access file:", e)
        
    return data, columns, selected_rows

# Tab2 FX resultaten     
tab2 = html.Div(
            [   html.H3('FX'),
                html.P('Show Daily FX Rates'),  
                html.Div (  
                    [
                        html.Div (
                            [ 
                                dct.DataTable( 
                                    id='tab2_datatable', 
                                    columns = [{"name": i, "id": i} for i in fx_cols], 
                                    data= df_gr.loc[(df_gr['curve_type']=='FX')][fx_cols].to_dict('records'),
                                    row_selectable='single',   
                                    selected_rows = [0],
                                    page_size=20,                                     
                                    style_cell_conditional = [ { 'if': {'column_id': c}, 'textAlign': 'left' } for c in fx_cols_alpha] ,
                                    # gestreepte regels voor de leesbaarheid                                
                                    style_data_conditional=[ { 'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)' }],                              
                                    # achtergrond kopregel
                                    style_header={'backgroundColor': 'rgb(230, 230, 230)','fontWeight': 'bold' },                                            
                                    style_as_list_view=True,
                                    sort_action="native",
                                    sort_mode="single"                                     
                                )
                            ], 
                            style={'display': 'inline-block','width': '49%','height': '10px' }
                        ),
                        html.Div( dcc.Graph(id="tab2_line_chart"), 
                                  style = {'display': 'inline-block','width': '40%','height': '10px'}
                                )                                   
                    ]
                )  
            ], 
    style={'display': 'inline-block', 'width': '85%','padding-left':'2%', 'padding-right':'2%'}            
    )

#Callback FX line chart
@app.callback(
     Output("tab2_line_chart", "figure"), 
     Input("tab2_datatable", "data"),
     Input("tab2_datatable", "selected_rows"),    
)
def update_line_chart(data, selected_row):
    if selected_row:
        ccy = data[selected_row[0]]['ccy']    
        
        df_fx_hist = df.loc[(df['curve_type']=='FX') & (df['rate_type']!='Spot End of Day')]        
        mask = df_fx_hist['ccy']==ccy                        
        title = 'Daily FX Rates ' + ccypair(ccy)               
        
        fig = linechart(df_fx_hist[mask],
                        'rate_dt',
                        'mid',
                        title,
                        "Rate Date",
                        "Daily Mid Close Rate",
                        None,
                        'closest')
         
        return fig
    else:
        return {}

    
# Tab3: Interest Rates
tab3 = html.Div(
    [
        html.H3('Interest Rates'),
        html.P('Show Daily PAR Rates and history'),  
        html.Div(
            [
                html.Div(
                    [
                        dcc.Dropdown(
                            id='tab3_ddlist_yieldgroup',
                            options=[{'label': i, 'value': i} for i in df_ir['yield_group'].unique()],
                            value = 'Swap Rates (Bond Based)', #df_ir['yield_group'].iloc[-1],
                            clearable=False
                        )            
                    ], style={'width': '24%', 'display': 'inline-block'}),
                html.Div(
                    [
                       dcc.Dropdown(
                            id='tab3_ddlist_ccy',                            
                            clearable=False
                        )              
                    ], style={'width': '24%', 'display': 'inline-block'})            
            ]
        )
        ,        
        html.Div(id='tab3-output-container', 
                 children =
                    [
                        dcc.Graph(id='tab3-graph'),
                        html.Div(id = 'tab3-hist_containter',
                                 children = 
                                     [                                        
                                        dcc.Graph(id='tab3-graph-hist')            
                                     ], 
                                     style={'display': 'none'}
                        )
                    ]
        )        
    ]
)


@app.callback(
    Output('tab3-hist_containter','style'),
    Input ('tab3-graph','hoverData')  
)
def hide_block(hover_data):
    
    # Hide history graph if nothing is selected.
    if hover_data is None or hover_data == []:
        return {'display': 'none'}
    else:
        return {'display': 'block'}
        
@app.callback(
    Output('tab3_ddlist_ccy','options'),
    Output('tab3_ddlist_ccy','value'),
    Input('tab3_ddlist_yieldgroup','value'),
    State('tab3_ddlist_ccy','value')
)
def select_ccy(yield_group, ccy):
            
    # Selecteer meest actuele datum
    dff_ir = df_ir[df_ir['rate_dt']==df_ir['rate_dt'].max()]    
    
    # Drop down selectie van Yield Group
    if yield_group is not None:
        dff_ir = dff_ir[dff_ir['yield_group'] == yield_group]    
              
    # Indien Rates niet aanwezig zijn of nog geen ccy gekozen is wijzigen we de selectie naar EUR.
    if ccy is None:
        ccy = 'EUR'
    
    # Check of de gekozen ccy voorkomt in de lijst
    if len (dff_ir.query('ccy == "' + ccy + '"')) == 0:
        # Wanneer EUR ook niet voorkomt, nemen we de eerste ccy in de lijst
        if len (dff_ir.query('ccy =="EUR"')) != 0:            
            ccy = 'EUR'
        else:
            ccy = dff_ir['ccy'].iloc[0]          
        
    # Update selectie lijst en gekozen ccy   
    options_list = [{'label': i, 'value': i} for i in dff_ir['ccy'].unique()]
    
    return options_list , ccy
         
@app.callback(
    Output('tab3-graph', 'figure'),
    Input ('tab3_ddlist_yieldgroup','value'),
    Input ('tab3_ddlist_ccy','value')    
)
def display_graph(yield_group, ccy):
                
    if ccy is not None and yield_group is not None:
    
        # Selecteer meest actuele datum
        dff_ir = df_ir[df_ir['rate_dt']==df_ir['rate_dt'].max()]        

        # Drop down selectie van Yield Group en Ccy
        dff_ir = dff_ir[dff_ir['yield_group'] == yield_group]    
        dff_ir = dff_ir[dff_ir['ccy'] == ccy]

        # Graph Title
        title = 'Interest Yield ' + ccy + ' ' + yield_group
            
        fig = linechart(dff_ir,
                        'actual_dt',
                        'mid',
                        title,
                        "Actual Date",
                        "Daily Mid Close Rate", 
                        'rate_name', 
                        'x unified',
                        'time_band')
    else:
        fig = {}
        
    return fig



@app.callback(
    Output('tab3-graph-hist', 'figure'),
    Input ('tab3_ddlist_yieldgroup','value'),
    Input ('tab3_ddlist_ccy','value'),
    Input ('tab3-graph','hoverData'),
    State ('tab3-graph','figure')
)
def display_hist(yield_group, ccy, hoverData, figure):
                    
    if hoverData and ccy is not None and yield_group is not None and figure is not None: 
        
        dff_ir = df_ir
        
        # Time Band is needed to select the historical rates for data point x
        time_band = hoverData['points'][0]['customdata'][0]
                      
        if time_band:
            # Drop down selectie van Yield Group en Ccy
            dff_ir = dff_ir[dff_ir['yield_group'] == yield_group]    
            dff_ir = dff_ir[dff_ir['ccy'] == ccy]        
            dff_ir = dff_ir[dff_ir['time_band'] == time_band]
        
            title = 'Interest Yield history ' + ccy + ' ' + yield_group + ' ' + time_band 
            title += '<br>' + 'Historical view showing the movement of one data point in time.'
        
            fig = linechart(dff_ir,'rate_dt','mid', title,'Rate Date','Daily Mid Close Rate','rate_name', 'closest' )
            
            # Probleem met de kleuren: deze komen niet altijd overeen met de oorspronkelijke chart wat nogal verwarrend is           
            colordict = {}
            # Get colors van main graph
            for i in figure['data']:
                if 'legendgroup' in i:
                    rate_name = i['legendgroup']
                    if 'line' in i:
                        if 'color' in i['line']:
                            color = i['line']['color']                                
                            colordict[rate_name] = color           
            # Set colors based on the main graph
            if 'data' in fig:
                for i in fig['data']:
                    if 'legendgroup' in i:
                        rate_name = i['legendgroup']
                        if 'line' in i:
                            if 'color' in i['line']:
                                if rate_name in colordict:
                                    i['line']['color'] = colordict[rate_name]
                                                              
        else:
            fig = {}                   
    else:
        fig =  {}        
    return fig


# Tab4 Bonds
tab4 = html.Div(
            [   html.H3('Bond Prices'),
                html.P('Show Daily Bond Prices'),  
                html.Div (  
                    [
                        html.Div (
                            [ 
                                dct.DataTable( 
                                    id='tab4_datatable', 
                                    columns = [{"name": i, "id": i} for i in bond_cols], 
                                    data= df_gr.loc[(df_gr['curve_type']=='Bonds')][bond_cols].to_dict('records'),
                                    row_selectable='single',   
                                    selected_rows = [0],
                                    page_size=20,                                     
                                    style_cell_conditional = [ { 'if': {'column_id': c}, 'textAlign': 'left' } for c in bond_cols_alpha],
                                    # gestreepte regels voor de leesbaarheid                                
                                    style_data_conditional=[ { 'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)' }],
                                    # achtergrond kopregel
                                    style_header={'backgroundColor': 'rgb(230, 230, 230)','fontWeight': 'bold' },
                                    style_as_list_view=True,
                                    sort_action="native",
                                    sort_mode="single"                                    
                                )
                            ], 
                            style={'display': 'inline-block','width': '49%','height': '10px' }
                        ),
                        html.Div( dcc.Graph(id="tab4_line_chart"), 
                                  style = {'display': 'inline-block','width': '40%','height': '10px'}
                                )                                   
                    ]
                )  
            ], 
    style={'display': 'inline-block', 'width': '85%','padding-left':'2%', 'padding-right':'2%'}            
    )

#Callback Bond line chart
@app.callback(
     Output("tab4_line_chart", "figure"), 
     Input("tab4_datatable", "data"),
     Input("tab4_datatable", "selected_rows"),    
)
def update_bond_chart(data, selected_row):
    if selected_row:
        
        df_bonds_hist = df.loc[(df['curve_type']=='Bonds') ]        
        bondname = data[selected_row[0]]['rate_name']                
        mask = df_bonds_hist['rate_name']==bondname 
        title = 'Daily Bond Prices for ISIN ' + bondname
        
        fig = linechart(df_bonds_hist[mask],
                        'rate_dt',
                        'mid',
                        title,
                        "Rate Date",
                        "Daily Mid Close Rate",
                        None,
                        'closest')
         
        return fig
    else:
        return {}

# Run the App
app.run_server()

Dash app running on http://127.0.0.1:8050/
